<a href="https://colab.research.google.com/github/joiedevivre02/joiedevivre02/blob/main/Repeat_Users_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as numpy
import os
import datetime as dt

from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
# Import and check dataframe
rootpath = '/content/gdrive/MyDrive/superapp_interns/Assignment/JM'

df = pd.read_csv(os.path.join(rootpath, 'repeat_users.csv'), dtype={'number':'object'})
df['datetime'] = pd.to_datetime(df['datetime'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 30 to 29
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   number    32 non-null     object        
 1   datetime  33 non-null     datetime64[ns]
 2   service   33 non-null     object        
 3   result    33 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.3+ KB


In [ ]:
# Show different end result
df["result"].unique()

array(['system_cancelled', 'completed', 'patient_no_show'], dtype=object)

In [ ]:
"""
Cancellation Cases

Cases that have not yet tried again.
  Case#1: Get cancelled, with no history
  Case#2: Get cancelled, with most recent history of getting cancelled
  Case#3: Get cancelled, with most recent history of NOT getting cancelled

Cases that tries again after cancellation
  Case#4: Get cancelled, with no history
  Case#5: Get cancelled, with most recent history of getting cancelled
  Case#6: Get cancelled, with most recent history of NOT getting cancelled
"""

case_1 = []
case_2 = []
case_3 = []
case_4 = []
case_5 = []
case_6 = []

# Holds user(number) of having only one row (no_history)
no_history = []

# Group rows by user(number) then iterate over each group
for name, group in df.groupby("number"):

  # Reverse to start at most recent transaction
  group = group.sort_values('datetime', ascending=False, ignore_index=True)
  series = group["result"].reset_index(drop=True)

  # Case conditions
  if len(series) == 1 and series[0] == 'system_cancelled':
    case_1.append(name)
    continue

  for i in range(len(series)-1):
    if i!=len(series)-1:

      if series[i] == 'system_cancelled':
        if series[i+1] == 'system_cancelled' and i==0:
          case_2.append(name)
        elif series[i+1] != 'system_cancelled' and i==0:
          case_3.append(name)
        elif series[i+1] == 'system_cancelled' and i>0:
          case_5.append(name)
        elif series[i+1] != 'system_cancelled' and i>0:
          case_6.append(name)
        break

      elif series.iloc[-1] == 'system_cancelled':
          case_4.append(name)
          break

# Verify counts for each caes, must be total to the original number of rows
len(df["number"].unique()), len(num_users_with_cancel), len(case_1), len(case_2), len(case_3), len(case_4), len(case_5), len(case_6)

In [ ]:
# Convert to list
series = case_1, case_2, case_3, case_4, case_5, case_6
series = pd.Series([len(df["number"].unique()), len(case_1), len(case_2), len(case_3), len(case_4), len(case_5), len(case_6)])
series.index = ["unique", "with cancel", "case_1", "case_2", "case_3", "case_4", "case_5", "case_6"]

In [ ]:
# Display
series